# Healthcare No-Show Analytics Data Cleaning

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/Users/aaliyahgritly/Downloads/Bebo study items/(P5) Healthcare No Shows Project/Medical-Noshows-May-2016.csv')

In [4]:
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


In [6]:
df.groupby('Age')['No-show'].value_counts(ascending=False)

Age   No-show
-1    No            1
 0    No         2900
      Yes         639
 1    No         1858
      Yes         415
                 ... 
 99   No            1
 100  No            4
 102  No            2
 115  Yes           3
      No            2
Name: count, Length: 204, dtype: int64

In [7]:
df.groupby('Neighbourhood')['No-show'].value_counts()

Neighbourhood    No-show
AEROPORTO        No            7
                 Yes           1
ANDORINHAS       No         1741
                 Yes         521
ANTÔNIO HONÓRIO  No          221
                            ... 
TABUAZEIRO       Yes         573
UNIVERSITÁRIO    No          120
                 Yes          32
VILA RUBIM       No          710
                 Yes         141
Name: count, Length: 160, dtype: int64

Start cleaning process by standardizing column names

In [8]:
# Making column names consistent
df.columns = df.columns.str.strip().str.lower().str.replace("-", "_").str.replace(" ", "_")

Convert data types

In [9]:
# Changing date columns to datetime format
df['appointmentday'] = pd.to_datetime(df['appointmentday'])
df['scheduledday'] = pd.to_datetime(df['scheduledday'])

In [10]:
df['patientid'] = df['patientid'].astype('string') # Converting patient ID to string type

Create `waiting_days` column

In [11]:
# Firts, this will extract the date part from the datetime columns
df['scheduled_date'] = df['scheduledday'].dt.date 
df['appointment_date'] = df['appointmentday'].dt.date 

# Now, we can calculate the difference in days
df['waiting_days'] = (pd.to_datetime(df['appointment_date']) - pd.to_datetime(df['scheduled_date'])).dt.days

In [12]:
df['waiting_days'].describe() # The min value is negative, which is not possible, so we need to address that.

count    110527.000000
mean         10.183702
std          15.254996
min          -6.000000
25%           0.000000
50%           4.000000
75%          15.000000
max         179.000000
Name: waiting_days, dtype: float64

A small number of records showed appointment dates earlier than scheduled dates.   
These were treated as same-day appointments by setting waiting time to zero, aligning with real-world scheduling logic.

In [13]:
# These are the rows with negative waiting days
df[df['waiting_days'] < 0][['scheduledday', 'appointmentday', 'scheduled_date', 'appointment_date', 'waiting_days']].head(10)

,scheduledday,appointmentday,scheduled_date,appointment_date,waiting_days
27033,2016-05-10 10:51:53+00:00,2016-05-09 00:00:00+00:00,2016-05-10,2016-05-09,-1
55226,2016-05-18 14:50:41+00:00,2016-05-17 00:00:00+00:00,2016-05-18,2016-05-17,-1
64175,2016-05-05 13:43:58+00:00,2016-05-04 00:00:00+00:00,2016-05-05,2016-05-04,-1
71533,2016-05-11 13:49:20+00:00,2016-05-05 00:00:00+00:00,2016-05-11,2016-05-05,-6
72362,2016-05-04 06:50:57+00:00,2016-05-03 00:00:00+00:00,2016-05-04,2016-05-03,-1


In [14]:
df['waiting_days'] = pd.to_datetime(df['waiting_days'], errors='coerce')
df['appointment_date'] = pd.to_datetime(df['appointment_date'], errors='coerce')
df['scheduled_date'] = pd.to_datetime(df['scheduled_date'], errors='coerce')

In [15]:

df['waiting_days'] = (df['appointment_date'] - df['scheduled_date']).dt.days

In [16]:
df['scheduled_date'] = pd.to_datetime(df['scheduledday']).dt.normalize()
df['appointment_date'] = pd.to_datetime(df['appointmentday']).dt.normalize() # Normalizing to remove time component

df['waiting_days'] = (df['appointment_date'] - df['scheduled_date']).dt.days

In [17]:
# Setting negative waiting days to zero
df.loc[df['waiting_days'] < 0, 'waiting_days'] = 0

In [18]:
df['waiting_days'].describe() # Now the min value shows 0

count    110527.000000
mean         10.183792
std          15.254924
min           0.000000
25%           0.000000
50%           4.000000
75%          15.000000
max         179.000000
Name: waiting_days, dtype: float64

Create `appointment_weekday` and `appointment_month` columns

In [19]:
df['appointment_weekday'] = df['appointmentday'].dt.day_name() # Extracting day name from appointment date

df['appointment_month'] = df['appointmentday'].dt.month_name() # Extracting month name from appointment date

Create a `has_disability` column

In [20]:
df['has_disability'] = df['handcap'].apply(lambda x: 1 if x > 0 else 0) # 1 if patient has any disability, else 0

Replace negatives in `age` column

In [21]:
df.loc[df['age'] < 0, 'age'] = pd.NA # Replacing negative ages with NaN

In [22]:
df['age'].describe()

count    110526.000000
mean         37.089219
std          23.110026
min           0.000000
25%          18.000000
50%          37.000000
75%          55.000000
max         115.000000
Name: age, dtype: float64

Create column `no_show_flag` to make no-show less confusing

In [23]:
# Here, Yes = 1 (no-show), No = 0 (showed up)
df['no_show_flag'] = df['no_show'].map({'Yes': 1, 'No': 0})

In [24]:
df[['no_show', 'no_show_flag']].value_counts()

no_show  no_show_flag
No       0               88208
Yes      1               22319
Name: count, dtype: int64

Create age groups   
Child: 0-12   
Teen: 13-17   
Young Adult: 18-35   
Adult: 36-55   
Senior: 56-75   
Elderly: 76+

In [25]:
def categorize_age(age):
    if age < 0 or pd.isna(age):
        return 'Unknown'
    elif age <= 12:
        return 'Child (0-12)'
    elif age <= 17:
        return 'Teen (13-17)'
    elif age <= 35:
        return 'Young Adult (18-35)'  
    elif age <= 55:
        return 'Adult (36-55)' 
    elif age <= 75:
        return 'Senior (56-75)'
    else:
        return 'Elderly (76+)'

In [26]:
df['age_group'] = df['age'].apply(categorize_age) # Applying the function to create age groups

In [27]:
df['age_group'].value_counts()

age_group
Adult (36-55)          30019
Young Adult (18-35)    25624
Senior (56-75)         22112
Child (0-12)           21036
Teen (13-17)            6343
Elderly (76+)           5392
Unknown                    1
Name: count, dtype: int64

Create wait time buckets

In [28]:
def categorize_waiting_time(days):
    if pd.isna(days):
        return 'Unknown'
    elif days == 0:
        return 'Same Day'
    elif days <= 7:
        return '1-7 Days'
    elif days <= 14:
        return '8-14 Days'
    elif days <= 30:
        return '15-30 Days'
    else:
        return '30+ Days'

In [29]:
df['wait_time_group'] = df['waiting_days'].apply(categorize_waiting_time) # Applying the function to waiting_days column to create waiting time groups

In [30]:
df[['wait_time_group']].value_counts()

wait_time_group
Same Day           38568
1-7 Days           32185
15-30 Days         17371
8-14 Days          12025
30+ Days           10378
Name: count, dtype: int64

Create a high risk no-show flag

In [31]:
df['high_risk_no_show'] = (df['waiting_days'] > 14) & (df['sms_received'] == 0).astype(int) # if they did not receive SMS and waiting days > 14, then high risk no show

In [32]:
df['high_risk_no_show'].value_counts()

high_risk_no_show
False    99708
True     10819
Name: count, dtype: int64

Now to do some sanity checks

In [33]:
overall_no_show = df['no_show_flag'].mean()
print(f"Overall no-show rate: {overall_no_show:.2%}")

Overall no-show rate: 20.19%


In [34]:
sms_no_show = df.groupby('sms_received')['no_show_flag'].mean() 
print(sms_no_show)

# Maybe hospitals send SMS more often to patients who are already at higher risk of missing (older patients, long wait times, first-time patients)

# 16.7% no-show rate for those who did not receive SMS
# 27.6% no-show rate for those who received SMS

sms_received
0    0.167033
1    0.275745
Name: no_show_flag, dtype: float64


In [34]:
age_no_show = df.groupby('age_group')['no_show_flag'].mean().sort_index()
print(age_no_show)

# Adults: 19.7% no-show rate
# Young Adults: 23.8% no-show rate
# Seniors: 15.5% no-show rate
# Children: 20.5% no-show rate
# Elderly: 16.1% no-show rate
# Teens: 26.6% no-show rate

age_group
Adult (36-55)          0.197075
Child (0-12)           0.204744
Elderly (76+)          0.161350
Senior (56-75)         0.155255
Teen (13-17)           0.266435
Unknown                0.000000
Young Adult (18-35)    0.238175
Name: no_show_flag, dtype: float64


In [34]:
wait_no_show = df.groupby('wait_time_group')['no_show_flag'].mean().sort_index()
print(wait_no_show)

# Same Day: 4.7% no-show rate
# 1-7 Days: 24.1% no-show rate
# 8-14 Days: 30.5% no-show rate
# 15-30 Days: 32.6% no-show rate
# 30+ Days: 33% no-show rate

# Longer wait times are associated with higher no-show rates

wait_time_group
1-7 Days      0.241479
15-30 Days    0.325888
30+ Days      0.330025
8-14 Days     0.304699
Same Day      0.046593
Name: no_show_flag, dtype: float64


In [35]:
risk_no_show = df.groupby('high_risk_no_show')['no_show_flag'].mean()
print(risk_no_show)

high_risk_no_show
False    0.183576
True     0.371106
Name: no_show_flag, dtype: float64


In [36]:
gender_no_show = df.groupby('gender')['no_show_flag'].mean()
print(gender_no_show)

gender
F    0.203146
M    0.199679
Name: no_show_flag, dtype: float64


Rename some columns for clarity

In [35]:
df.rename(columns = {'patientid': 'patient_id', 'appointmentid': 'appointment_id', 
                     'scheduledday': 'scheduled_day', 'appointmentday': 'appointment_day'}, inplace = True)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype              
---  ------               --------------   -----              
 0   patient_id           110527 non-null  string             
 1   appointment_id       110527 non-null  int64              
 2   gender               110527 non-null  object             
 3   scheduled_day        110527 non-null  datetime64[ns, UTC]
 4   appointment_day      110527 non-null  datetime64[ns, UTC]
 5   age                  110526 non-null  float64            
 6   neighbourhood        110527 non-null  object             
 7   scholarship          110527 non-null  int64              
 8   hipertension         110527 non-null  int64              
 9   diabetes             110527 non-null  int64              
 10  alcoholism           110527 non-null  int64              
 11  handcap              110527 non-null  int64              
 12  sm

In [37]:
df.head()

,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,...,scheduled_date,appointment_date,waiting_days,appointment_weekday,appointment_month,has_disability,no_show_flag,age_group,wait_time_group,high_risk_no_show
0,29872499824296.0,5642903,F,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,62.0,JARDIM DA PENHA,0,1,0,...,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,0,Friday,April,0,0,Senior (56-75),Same Day,False
1,558997776694438.0,5642503,M,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,56.0,JARDIM DA PENHA,0,0,0,...,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,0,Friday,April,0,0,Senior (56-75),Same Day,False
2,4262962299951.0,5642549,F,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,62.0,MATA DA PRAIA,0,0,0,...,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,0,Friday,April,0,0,Senior (56-75),Same Day,False
3,867951213174.0,5642828,F,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8.0,PONTAL DE CAMBURI,0,0,0,...,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,0,Friday,April,0,0,Child (0-12),Same Day,False
4,8841186448183.0,5642494,F,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,56.0,JARDIM DA PENHA,0,1,1,...,2016-04-29 00:00:00+00:00,2016-04-29 00:00:00+00:00,0,Friday,April,0,0,Senior (56-75),Same Day,False


Import the cleaned version of the data into a csv

In [38]:
df.to_csv('/Users/aaliyahgritly/Downloads/Bebo study items/(P5) Healthcare No Shows Project/Medical-Noshows-May-2016-Cleaned.csv', index=False)

In [ ]:
pd.set_option('display.max_columns', None)
df.sort_values('appointment_id').head()


,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,...,scheduled_date,appointment_date,waiting_days,appointment_weekday,appointment_month,has_disability,no_show_flag,age_group,wait_time_group,high_risk_no_show
3764,832256398961987.0,5030230,F,2015-11-10 07:13:56+00:00,2016-05-04 00:00:00+00:00,51.0,RESISTÊNCIA,0,0,0,...,2015-11-10 00:00:00+00:00,2016-05-04 00:00:00+00:00,176,Wednesday,May,0,0,Adult (36-55),30+ Days,False
46292,91637474953513.0,5122866,M,2015-12-03 08:17:28+00:00,2016-05-02 00:00:00+00:00,34.0,VILA RUBIM,0,1,0,...,2015-12-03 00:00:00+00:00,2016-05-02 00:00:00+00:00,151,Monday,May,0,1,Young Adult (18-35),30+ Days,False
102795,1216586867796.0,5134197,F,2015-12-07 10:40:59+00:00,2016-06-03 00:00:00+00:00,27.0,SÃO CRISTÓVÃO,1,0,0,...,2015-12-07 00:00:00+00:00,2016-06-03 00:00:00+00:00,179,Friday,June,0,1,Young Adult (18-35),30+ Days,False
102797,31899595421534.0,5134220,F,2015-12-07 10:42:42+00:00,2016-06-03 00:00:00+00:00,48.0,MARUÍPE,0,1,1,...,2015-12-07 00:00:00+00:00,2016-06-03 00:00:00+00:00,179,Friday,June,0,0,Adult (36-55),30+ Days,False
102796,9582232334148.0,5134223,F,2015-12-07 10:43:01+00:00,2016-06-03 00:00:00+00:00,80.0,SÃO CRISTÓVÃO,0,1,1,...,2015-12-07 00:00:00+00:00,2016-06-03 00:00:00+00:00,179,Friday,June,0,0,Elderly (76+),30+ Days,False
